In [1]:
#| default_exp testing

%pip install -q langchain_core
%pip install -q langchain_openai
%pip install -q nbdev

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_core.prompts import PromptTemplate
from hn_jobs_chat.util import loadJSON

import json

examplePrompt = PromptTemplate.from_template("""Here is an example job post:\n\n{job_posting}\n\nHere is the response to return for this example:\n\n{response}\n\n----------\n\n""")

def getJsonExamples():
    examplePrompts = []
    posts = loadJSON('posts.json')
    example_responses = loadJSON('./examples.json')

    for example, post in zip(example_responses, posts):
        examplePresentation = examplePrompt.format(job_posting=post['comment'], response=json.dumps(example))

        examplePrompts.append(examplePresentation)

    return examplePrompts

In [61]:
from langchain_core.prompts import PromptTemplate

subsequentPrompt = PromptTemplate.from_template("""
You will be returning one or more JSON objects together in an array to describe the jobs in a job posting.

Here are examples of the format of the JSON object you should return. For each example, first the job posting is displayed, then the data that should be extracted:\n\n{json_examples}\n

Here are the key values currently being used to categorize jobs, with a description of the contents it will hold:\n\n{key_values}.\n\n
* As you parse the job listing, for each piece of information you want to record, walk step by step through these key values, determining for each one where it is an appropriate key for the information you need to add. Find the most relevant key, do not add any new keys not on this list. If there is no relevant key, add the information to the 'additional_notes' array.
* If the information for a key isn't present, omit it from your response. \n
* If the post states that certain specific information is available from their website, mark that as 'ON WEBSITE'. Do not use this key unless a url is available, and the job listing explicitly says that the information is there.\n
* If there is only one job posting presented by the post, return a JSON object containing a summary of every separate piece of information in the job posting.\n* If the job describes multiple roles, create a separate JSON object for each role.\n* If the job states that multiple roles are open, but does not state the roles, use 'Multiple Roles' as the job title.\n\nHere is the job posting to evaluate:\n\n{job_posting}""")



In [4]:
from hn_jobs_chat.scraper import loadJSON

posts = loadJSON('posts.json')

In [5]:
# print(getJsonExamples())

In [6]:
# from langchain_openai import ChatOpenAI

# post = posts[0]

# msg = firstPrompt.format(job_posting = post['comment'])

# model = ChatOpenAI()

# response = model.invoke(msg)

In [3]:
from hn_jobs_chat.responseKeys import keysStr

keysStr = keys


In [8]:
import json

json_examples = getJsonExamples()
examplesStr = "".join(json_examples)

renderedStr = print(examplesStr)


Here is an example job post:

At Irth Solutions we build software that improve resilience and reduce risk to critical infrastructure that millions of people and businesses rely on each day. Our systems handle millions of requests a day, coordinating activities between critical infrastructure providers, contract locators, excavators, and a variety of other stakeholders throughout North America.
We’re growing our team and are in immediate need for a Senior Software Engineer. Here are some notes on the technical requirements:
* C# development experience using .NET Core, WebAPI, and Entity Framework
* Experience building browser applications using Angular and Typescript
* Strong relational database experience with Postgres (preferred) or MS SQL Server including SQL, database functions, indexing, and performance tuning
* Kubernetes, Docker, Linux
Apply here: https://apply.workable.com/irth-solutions/j/33AD66BA3A/
We’re growing, so new positions will open up frequently. Please check back wit

In [54]:
# |export
import json

def toValidJSON(response, repromptNum=None):
    # try catch here. If item fails, rerun it. If it fails more than 3 times, return None
    try:
        return json.loads(response.replace('\n',''))
    except:
        print('Json parsing failed on response', response)
        if repromptNum == None:
            repromptNum = 0
        
        if repromptNum > 3:
            return '{ "errpr": "Could not receive valid json: Too many reprompts","item": '+item+'}'
        
        repromptNum += 1
        return reprompt(response, repromptNum)
    
def reprompt(item, repromptNum=None):
    newPrompt = 'Your response: \n\n'+item.text+'was not valid json. Please try again. Here is your response. Please fix any json formatting errors. The output should be one or more json objects in an array.:\n\n' + item

    response = getResponse(newPrompt)

    print('response')
    return toJson(response['text'], repromptNum=repromptNum)


In [74]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
parsed_data = []

postItems = posts[:2]

for i in range(len(postItems)):
        msg = subsequentPrompt.format(job_posting = postItems[i]['comment'], json_examples = renderedStr, key_values = keyStr)
        
        resp = model.invoke(msg)

        parsed_datum = toValidJSON(resp.content)

        for  job in  parsed_datum:
            parsed_data.append(job)

            # initially I let the llm add any keys it was missing.
            #  This proved problematic as it would not use existing keys.
            # # We add any new keys to the permitted keys. 
            # # This does mean that the later prompts are potentially receive a different set of
            # # suggested keys, but those keys should only be added if necessary.
            # for jobKey in job.keys():
            #     keys.append(jobKey)
            #     keys = list(set(keys))
            #     keyStr = '\n'.join(keys)

print(parsed_data)

[{'company': 'Irth Solutions', 'job_title': 'Senior Software Engineer', 'job_description': 'Build software that improves resilience and reduce risk to critical infrastructure that millions of people and businesses rely on each day. Systems handle millions of requests a day, coordinating activities between critical infrastructure providers, contract locators, excavators, and other stakeholders throughout North America.', 'tech_stack': 'C# development experience using .NET Core, WebAPI, and Entity Framework. Experience building browser applications using Angular and Typescript. Strong relational database experience with Postgres (preferred) or MS SQL Server including SQL, database functions, indexing, and performance tuning. Kubernetes, Docker, Linux.', 'contact_url': 'https://apply.workable.com/irth-solutions/j/33AD66BA3A/'}, {'company': 'Tandem Health', 'job_title': 'Software Engineer', 'location': 'Stockholm, Sweden', 'employment_type': 'Full-time', 'job_description': 'Building a clin

We split the iteration over the request so we don't do it unnecessarily

In [75]:
from hn_jobs_chat.util import saveJSON

saveJSON('postData-2.json', parsed_data)
saveJSON('keys.json', keys)


In [ ]:

# let's see how frequently each key appears
# tally = {}
# keys
# for key in keys:
# #     result[key] = [datum[key] for datum in parsed_data]
#     tally[key] = sum([1 for datum in parsed_data if key in datum])

# # result, tally

We add all the gathered keys to the table.

We create a SQL query that posts by our new keys

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()